# Outward Bound ODK Python Analytics Notebook
### Goals and Benefits of this Workbook
#### Goals
1. To import test data from a kobotoolbox form. (done)
2. To manipulate the data so it is fit for analysis using basic libraries or SQL (idea is to have code easily adapted for use by the backend/portal dev team). (done)
3. To calculate effect sizes and the Kronbach's Alpha. (in progress - Kronback's Alpha is being pursued)
4. To export the results as a series of formatted tables and charts. (presently out of scope) <br>

#### Benefits
1. Code is flexible and can be changed to accomodate findings in naming conventions, data types, and the data model.
2. Code automates data transformations and calculations that take roughly 3 hours per program using excel.

### Review of Requirements
Outward Bound's Brief can be found in the link below. Discussion Points Follow. <br>
https://docs.google.com/document/d/1-1jFaH5GXX0erM9Z5jblwqCkWoXNUJVUStU4zF0REgo/edit?usp=sharing <br>
* There are multiple aggregation levels for the results - by individual, by group, by program.
* Outcome to question mapping will vary by program and survey type - so program will need to consult a question/effect index (map or join in DB logic)
* Cohen's Effect Size compares the means of 2 samples. As such, the STDEV used in the calculation needs to be pooled (RMS). 
* Still unclear on R^2 - this workbook calculates it using the formulae provided in the brief. 
https://stats.stackexchange.com/questions/15749/how-to-report-effect-size-measures-r-and-r-squared-and-what-is-a-non-technical-e
* Reports frequently reference an industry standard on effect size of 0.34
* Cronbach's alpha is a requirement to test 'internal consistency' of answers to questions belonging to each item or outcome. A better explanation of this can be found here: https://www.youtube.com/watch?v=uXKnn0T6Cyw

### Method and Assumptions 
#### Method
1. Parse data and question:outcome index into a pandas dataframe
2. Transform the data so that each row represents a sample, retaining an index of sample, group, and email.
3. Merge the data and index together, so that outcome is available.
4. Check assumptions used in analysis
5. Create pivot tables for outcome calculations at three aggregation levels - email, group, and program.
6. Calculate Cohen's d for effect size and r^2 for each d
7. Export pivot tables for viz off platform. 

#### Assumptions
1. Data will always be a complete sample of your population (BIG assumption - not everybody will respond at 3 months).
2. Emails will be cleaned to be a consistent identifier across samples
**The negative impact from these assumptions can be largely avoided by reporting at the Group and Program levels**
* Some of these assumptions may be cleared up by the literature - eg http://journals.sagepub.com/doi/abs/10.3102/00346543067001043 (though this article is presently behind a paywall)

### Future Work
1. Implement Cronbach's alpha using either native python, or leveraging a library (e.g. Scikit or Statsmodel)
3. Automate the code to take dropped files within a directory and spit out resulting files in the same directory (py executable).
3. Explore using Bokeh or another library to create an interactive, browser based vizualisation of the data (may be cleaner than exported tables or plots) - Lead URL: https://spapas.github.io/2016/09/21/pandas-pivot-table-primer/.

# Code
## 1. Parse data and question:outcome index into a pandas dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
data = pd.read_csv('Community_Data.csv')
index = pd.read_csv('Community_Index.csv')

Wall time: 6.99 ms


In [3]:
# First look
data.head()

,sample,date,group,gender,religion,age,firstName,lastName,email,QC01,...,QC21,QC22,QC23,QC24,QC25,__version__,_version_,_id,_uuid,_index
0,1,2017-08-24,group1,female,upsidedown,10,Eleven,Unsure,eleven@research.com,4,...,4,5,4,6,1,vvExt3a3agKiVoH6odd7Sh,vYFWVbHaUGhJsXuGETeoz8,6902251,918358fe-8970-40d0-ba15-2894a519be50,1
1,2,2017-08-26,group1,female,upsidedown,10,Eleven,Unsure,eleven@research.com,5,...,5,6,5,7,5,vvExt3a3agKiVoH6odd7Sh,vYFWVbHaUGhJsXuGETeoz8,6902251,918358fe-8970-40d0-ba15-2894a519be51,1
2,3,2017-11-26,group1,female,upsidedown,10,Eleven,Unsure,eleven@research.com,6,...,5,6,5,7,6,vvExt3a3agKiVoH6odd7Sh,vYFWVbHaUGhJsXuGETeoz8,6902251,918358fe-8970-40d0-ba15-2894a519be52,1
3,1,2017-08-24,group1,male,DND,9,Will,Byers,WillByers@test.com,4,...,4,4,4,4,4,vvExt3a3agKiVoH6odd7Sh,vYFWVbHaUGhJsXuGETeoz8,6902251,918358fe-8970-40d0-ba15-2894a519be53,1
4,2,2017-08-26,group1,male,DND,9,Will,Byers,WillByers@test.com,4,...,4,4,4,4,4,vvExt3a3agKiVoH6odd7Sh,vYFWVbHaUGhJsXuGETeoz8,6902251,918358fe-8970-40d0-ba15-2894a519be54,1


In [4]:
index.head()

,QID,OUTCOME
0,QC01,General Self Concept
1,QC02,Social Self Concept
2,QC03,Resilience (Recovery from Setbacks)
3,QC04,Intelligent Self Concept
4,QC05,Respect Personal Boundaries


In [5]:
# General data cleaning
# Normalize case and spacing for 'fields' in data frames
data.columns = [c.replace(' ', '_') for c in  data.columns]
data.columns =  [c.lower() for c in  data.columns]

index.columns = [c.replace(' ', '_') for c in  index.columns]
index.columns =  [c.lower() for c in  index.columns]

# Normalize index column values to match normazli
index['qid']=index['qid'].str.lower()

In [6]:
# column index
print("data",list(data.columns),"\n")
print("index",list(index.columns))
# for data, note that columns starting with qc are questions, values of scores.
# for the index, note that qid corresponds to question column labels

data ['sample', 'date', 'group', 'gender', 'religion', 'age', 'firstname', 'lastname', 'email', 'qc01', 'qc02', 'qc03', 'qc04', 'qc05', 'qc06', 'qc07', 'qc08', 'qc09', 'qc10', 'qc11', 'qc12', 'qc13', 'qc14', 'qc15', 'qc16', 'qc17', 'qc18', 'qc19', 'qc20', 'qc21', 'qc22', 'qc23', 'qc24', 'qc25', '__version__', '_version_', '_id', '_uuid', '_index'] 

index ['qid', 'outcome']


## 2. Transform the data so that each row represents a sample, whilst retaining the index of sample, group, and email.

In [7]:
question_cols = [col for col in data.columns if 'qc' in col]
junk_cols = ['__version__','_version_','_id','_uuid']
index_cols = list(set(list(data.columns))-set(question_cols)-set(junk_cols))

In [8]:
# Double check the index
print(index_cols)

['sample', '_index', 'date', 'lastname', 'gender', 'firstname', 'religion', 'email', 'group', 'age']


In [9]:
# Melt is a Pandas alternative to the UNPIVOT SQL operation
data2 = pd.melt(data,id_vars=index_cols,value_vars=question_cols,var_name='qid',value_name='score')

In [10]:
# Count rows and columns before and after transformation
print(data.shape,data2.shape)

(21, 39) (525, 12)


## 3. Merge the data and index together, so that outcome is available.

In [11]:
data3 = data2.merge(index,how='left',on='qid')

In [12]:
# Take a look at the transformed table (integrity check)
data3.head()

,sample,_index,date,lastname,gender,firstname,religion,email,group,age,qid,score,outcome
0,1,1,2017-08-24,Unsure,female,Eleven,upsidedown,eleven@research.com,group1,10,qc01,4,General Self Concept
1,2,1,2017-08-26,Unsure,female,Eleven,upsidedown,eleven@research.com,group1,10,qc01,5,General Self Concept
2,3,1,2017-11-26,Unsure,female,Eleven,upsidedown,eleven@research.com,group1,10,qc01,6,General Self Concept
3,1,1,2017-08-24,Byers,male,Will,DND,WillByers@test.com,group1,9,qc01,4,General Self Concept
4,2,1,2017-08-26,Byers,male,Will,DND,WillByers@test.com,group1,9,qc01,4,General Self Concept


In [13]:
data3.dtypes

sample        int64
_index        int64
date         object
lastname     object
gender       object
firstname    object
religion     object
email        object
group        object
age           int64
qid          object
score         int64
outcome      object
dtype: object

## 4. Assumption Check:
Assumption 1 is: <br>
Data will always be a complete sample of your population (BIG assumption - not everybody will respond at 3 months). <br>
This code checks whether it holds for the input data.

In [14]:
# Equivalence test - do we have complete data between samples 1 and 2, and between samples 2 and 3?
print("Sample 2 vs 1 - difference in samples (should be zero):" + "\n" + str(data3[(data3['sample']==2)].outcome.value_counts() - data3[(data3['sample']==1)].outcome.value_counts()) + "\n")
print("Sample 3 vs 2 - difference in samples (should be zero):" + "\n" + str(data3[(data3['sample']==3)].outcome.value_counts() - data3[(data3['sample']==2)].outcome.value_counts()))

Sample 2 vs 1 - difference in samples (should be zero):
Resilience (Recovery from Setbacks)    0
Social Self Concept                    0
Intelligent Self Concept               0
Respect Personal Boundaries            0
General Self Concept                   0
Name: outcome, dtype: int64

Sample 3 vs 2 - difference in samples (should be zero):
Resilience (Recovery from Setbacks)    0
Social Self Concept                    0
Intelligent Self Concept               0
Respect Personal Boundaries            0
General Self Concept                   0
Name: outcome, dtype: int64


## 5. Create pivot tables for outcome calculations at three aggregation levels - email, group, and program.

In [15]:
# This leverages the Python pivot_table function, with the output of a pandas dataframe stored as a dictionary.
pivot_levels = ['email','group','_index'] #note that '_index' is chosen as a proxy to program, as it remains constant for all responses in the form
pivot_columns = ['sample']
pivot_aggfunc = [np.mean,np.std] # calculates mean, stdev, for each sample, at each aggregation level (so outcome, split by whatever)
pivot_index = ['outcome']
pivot_values = ['score']
pivot_tables = {}

In [16]:
for a in range(len(pivot_levels)):
    pivot_index.insert(0,pivot_levels[a])
    pivot_tables["table{0}".format(a)]=pd.pivot_table(data3,index=pivot_index,values=pivot_values,columns=pivot_columns,aggfunc=pivot_aggfunc)
    pivot_index = ['outcome']

In [17]:
pivot_key = list(pivot_tables.keys())

In [18]:
# display resulting tables
from IPython.display import display
for a in range(len(pivot_key)):
    display(pivot_tables[pivot_key[a]])

mean  \
                                                                 score   
sample                                                               1   
email                    outcome                                         
DustinHenderson@test.com General Self Concept                 3.000000   
                         Intelligent Self Concept             6.000000   
                         Resilience (Recovery from Setbacks)  3.666667   
                         Respect Personal Boundaries          5.000000   
                         Social Self Concept                  2.200000   
LukeSinclair@test.com    General Self Concept                 4.750000   
                         Intelligent Self Concept             3.400000   
                         Resilience (Recovery from Setbacks)  3.666667   
                         Respect Personal Boundaries          5.000000   
                         Social Self Concept                  4.400000   
MadMax_Hayfield@test.com General Self Concept                 3.750000   
                         Intelligent Self Concept             4.400000   
                         Resilience (Recovery from Setbacks)  2.500000   
                         Respect Personal Boundaries          3.800000   
                         Social Self Concept                  3.200000   
MikeWheeler@test.com     General Self Concept                 2.000000   
                         Intelligent Self Concept             5.200000   
                         Resilience (Recovery from Setbacks)  2.000000   
                         Respect Personal Boundaries          3.600000   
                         Social Self Concept                  2.400000   
SteveHarrington@test.com General Self Concept                 6.250000   
                         Intelligent Self Concept             3.200000   
                         Resilience (Recovery from Setbacks)  2.500000   
                         Respect Personal Boundaries          4.000000   
                         Social Self Concept                  7.000000   
WillByers@test.com       General Self Concept                 4.000000   
                         Intelligent Self Concept             4.000000   
                         Resilience (Recovery from Setbacks)  4.000000   
                         Respect Personal Boundaries          4.000000   
                         Social Self Concept                  4.000000   
eleven@research.com      General Self Concept                 3.750000   
                         Intelligent Self Concept             7.200000   
                         Resilience (Recovery from Setbacks)  5.500000   
                         Respect Personal Boundaries          2.200000   
                         Social Self Concept                  1.000000   

                                                                        \
                                                                         
sample                                                               2   
email                    outcome                                         
DustinHenderson@test.com General Self Concept                 5.500000   
                         Intelligent Self Concept             6.000000   
                         Resilience (Recovery from Setbacks)  5.500000   
                         Respect Personal Boundaries          6.000000   
                         Social Self Concept                  4.800000   
LukeSinclair@test.com    General Self Concept                 6.250000   
                         Intelligent Self Concept             5.200000   
                         Resilience (Recovery from Setbacks)  5.500000   
                         Respect Personal Boundaries          5.400000   
                         Social Self Concept                  5.800000   
MadMax_Hayfield@test.com General Self Concept                 5.250000   
                         Intelligent Self Concept             5.00000

mean                      \
                                               score                       
sample                                             1         2         3   
group  outcome                                                             
group1 General Self Concept                 3.928571  5.607143  5.964286   
       Intelligent Self Concept             4.771429  5.400000  5.971429   
       Resilience (Recovery from Setbacks)  3.404762  5.333333  5.690476   
       Respect Personal Boundaries          3.942857  5.142857  5.457143   
       Social Self Concept                  3.457143  5.085714  5.514286   

                                                 std                      
                                               score                      
sample                                             1         2         3  
group  outcome                                                            
group1 General Self Concept                 1.961265  1.314852  1.400586  
       Intelligent Self Concept             1.895594  1.683833  1.339133  
       Resilience (Recovery from Setbacks)  1.326273  1.096928  1.047368  
       Respect Personal Boundaries          1.607754  1.061155  0.980482  
       Social Self Concept                  2.146719  1.336621  1.172514

mean                      \
                                               score                       
sample                                             1         2         3   
_index outcome                                                             
1      General Self Concept                 3.928571  5.607143  5.964286   
       Intelligent Self Concept             4.771429  5.400000  5.971429   
       Resilience (Recovery from Setbacks)  3.404762  5.333333  5.690476   
       Respect Personal Boundaries          3.942857  5.142857  5.457143   
       Social Self Concept                  3.457143  5.085714  5.514286   

                                                 std                      
                                               score                      
sample                                             1         2         3  
_index outcome                                                            
1      General Self Concept                 1.961265  1.314852  1.400586  
       Intelligent Self Concept             1.895594  1.683833  1.339133  
       Resilience (Recovery from Setbacks)  1.326273  1.096928  1.047368  
       Respect Personal Boundaries          1.607754  1.061155  0.980482  
       Social Self Concept                  2.146719  1.336621  1.172514

In [19]:
# check index on resulting tables - are they the same? (assumption used in calculating cohen's d)
for a in range(len(pivot_key)):
    print(pivot_tables[pivot_key[a]].columns)

MultiIndex(levels=[['mean', 'std'], ['score'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 1, 2, 0, 1, 2]],
           names=[None, None, 'sample'])
MultiIndex(levels=[['mean', 'std'], ['score'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 1, 2, 0, 1, 2]],
           names=[None, None, 'sample'])
MultiIndex(levels=[['mean', 'std'], ['score'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 1, 2, 0, 1, 2]],
           names=[None, None, 'sample'])


## 6. Calculate Cohen's d for effect size, and r^2 for percentage of change

In [20]:
# Calculations are between samples 1 and 2, and 2 and 3, using pivot calculations and pooled stdev. 
# i.e. https://sisu.ut.ee/sites/default/files/measurement/files/pooled_standard_deviation.pdf 
# note that the following language exploits python object mutability - SQL may need to be more specific.

In [21]:
# Calculate Effect between samples 1 and 2 (i.e. immediately after the trip):
for a in range(len(pivot_key)):
    table = pivot_tables[pivot_key[a]]
    table['effect_immediate']=(table[('mean','score',2)] - table[('mean','score',1)])/ \
    ((table[('std','score',2)]**2 + table[('std','score',1)]**2)/2)**0.5

In [22]:
# Calculate Effect between samples 2 and 3 (i.e. 3 months after the trip):
for a in range(len(pivot_key)):
    table = pivot_tables[pivot_key[a]]
    table['effect_3months']=(table[('mean','score',3)] - table[('mean','score',2)])/ \
    ((table[('std','score',3)]**2 + table[('std','score',2)]**2)/2)**0.5

In [23]:
# Calculate r^2 between samples 1 and 2, and 2 and 3 (i.e. immediately after the trip, and 3 months after the trip):
for a in range(len(pivot_key)):
    table = pivot_tables[pivot_key[a]]
    table['r_square_immediate'] = table['effect_immediate']**2/(4+table['effect_immediate']**2)
    table['r_square_3months'] = table['effect_3months']**2/(4+table['effect_3months']**2)

In [24]:
# display resulting tables
from IPython.display import display
for a in range(len(pivot_key)):
    display(pivot_tables[pivot_key[a]])

mean  \
                                                                 score   
sample                                                               1   
email                    outcome                                         
DustinHenderson@test.com General Self Concept                 3.000000   
                         Intelligent Self Concept             6.000000   
                         Resilience (Recovery from Setbacks)  3.666667   
                         Respect Personal Boundaries          5.000000   
                         Social Self Concept                  2.200000   
LukeSinclair@test.com    General Self Concept                 4.750000   
                         Intelligent Self Concept             3.400000   
                         Resilience (Recovery from Setbacks)  3.666667   
                         Respect Personal Boundaries          5.000000   
                         Social Self Concept                  4.400000   
MadMax_Hayfield@test.com General Self Concept                 3.750000   
                         Intelligent Self Concept             4.400000   
                         Resilience (Recovery from Setbacks)  2.500000   
                         Respect Personal Boundaries          3.800000   
                         Social Self Concept                  3.200000   
MikeWheeler@test.com     General Self Concept                 2.000000   
                         Intelligent Self Concept             5.200000   
                         Resilience (Recovery from Setbacks)  2.000000   
                         Respect Personal Boundaries          3.600000   
                         Social Self Concept                  2.400000   
SteveHarrington@test.com General Self Concept                 6.250000   
                         Intelligent Self Concept             3.200000   
                         Resilience (Recovery from Setbacks)  2.500000   
                         Respect Personal Boundaries          4.000000   
                         Social Self Concept                  7.000000   
WillByers@test.com       General Self Concept                 4.000000   
                         Intelligent Self Concept             4.000000   
                         Resilience (Recovery from Setbacks)  4.000000   
                         Respect Personal Boundaries          4.000000   
                         Social Self Concept                  4.000000   
eleven@research.com      General Self Concept                 3.750000   
                         Intelligent Self Concept             7.200000   
                         Resilience (Recovery from Setbacks)  5.500000   
                         Respect Personal Boundaries          2.200000   
                         Social Self Concept                  1.000000   

                                                                        \
                                                                         
sample                                                               2   
email                    outcome                                         
DustinHenderson@test.com General Self Concept                 5.500000   
                         Intelligent Self Concept             6.000000   
                         Resilience (Recovery from Setbacks)  5.500000   
                         Respect Personal Boundaries          6.000000   
                         Social Self Concept                  4.800000   
LukeSinclair@test.com    General Self Concept                 6.250000   
                         Intelligent Self Concept             5.200000   
                         Resilience (Recovery from Setbacks)  5.500000   
                         Respect Personal Boundaries          5.400000   
                         Social Self Concept                  5.800000   
MadMax_Hayfield@test.com General Self Concept                 5.250000   
                         Intelligent Self Concept             5.00000

mean                      \
                                               score                       
sample                                             1         2         3   
group  outcome                                                             
group1 General Self Concept                 3.928571  5.607143  5.964286   
       Intelligent Self Concept             4.771429  5.400000  5.971429   
       Resilience (Recovery from Setbacks)  3.404762  5.333333  5.690476   
       Respect Personal Boundaries          3.942857  5.142857  5.457143   
       Social Self Concept                  3.457143  5.085714  5.514286   

                                                 std                      \
                                               score                       
sample                                             1         2         3   
group  outcome                                                             
group1 General Self Concept                 1.961265  1.314852  1.400586   
       Intelligent Self Concept             1.895594  1.683833  1.339133   
       Resilience (Recovery from Setbacks)  1.326273  1.096928  1.047368   
       Respect Personal Boundaries          1.607754  1.061155  0.980482   
       Social Self Concept                  2.146719  1.336621  1.172514   

                                           effect_immediate effect_3months  \
                                                                             
sample                                                                       
group  outcome                                                               
group1 General Self Concept                        1.005349       0.262915   
       Intelligent Self Concept                    0.350600       0.375624   
       Resilience (Recovery from Setbacks)         1.584673       0.333021   
       Respect Personal Boundaries                 0.880958       0.307636   
       Social Self Concept                         0.910758       0.340881   

                                           r_square_immediate r_square_3months  
                                                                                
sample                                                                          
group  outcome                                                                  
group1 General Self Concept                          0.201713         0.016988  
       Intelligent Self Concept                      0.029814         0.034072  
       Resilience (Recovery from Setbacks)           0.385673         0.026978  
       Respect Personal Boundaries                   0.162494         0.023113  
       Social Self Concept                           0.171753         0.028230

mean                      \
                                               score                       
sample                                             1         2         3   
_index outcome                                                             
1      General Self Concept                 3.928571  5.607143  5.964286   
       Intelligent Self Concept             4.771429  5.400000  5.971429   
       Resilience (Recovery from Setbacks)  3.404762  5.333333  5.690476   
       Respect Personal Boundaries          3.942857  5.142857  5.457143   
       Social Self Concept                  3.457143  5.085714  5.514286   

                                                 std                      \
                                               score                       
sample                                             1         2         3   
_index outcome                                                             
1      General Self Concept                 1.961265  1.314852  1.400586   
       Intelligent Self Concept             1.895594  1.683833  1.339133   
       Resilience (Recovery from Setbacks)  1.326273  1.096928  1.047368   
       Respect Personal Boundaries          1.607754  1.061155  0.980482   
       Social Self Concept                  2.146719  1.336621  1.172514   

                                           effect_immediate effect_3months  \
                                                                             
sample                                                                       
_index outcome                                                               
1      General Self Concept                        1.005349       0.262915   
       Intelligent Self Concept                    0.350600       0.375624   
       Resilience (Recovery from Setbacks)         1.584673       0.333021   
       Respect Personal Boundaries                 0.880958       0.307636   
       Social Self Concept                         0.910758       0.340881   

                                           r_square_immediate r_square_3months  
                                                                                
sample                                                                          
_index outcome                                                                  
1      General Self Concept                          0.201713         0.016988  
       Intelligent Self Concept                      0.029814         0.034072  
       Resilience (Recovery from Setbacks)           0.385673         0.026978  
       Respect Personal Boundaries                   0.162494         0.023113  
       Social Self Concept                           0.171753         0.028230

## 7. Export pivot tables for viz off platform. 

In [25]:
for key, value in pivot_tables.items():
    value.to_csv(key + '.csv')
# Note - might make sense to re-index and recut so we have clean tables of level and effect - i.e. email and effect_immediate or effect_3month